<a href="https://colab.research.google.com/github/dahiyajoy/CV-Data-Extraction-Model/blob/Master/Speech_Diarization_Final.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Speech Diarizartion**

In [2]:
from google.colab import drive
drive.mount('/content/drive')

file_path = '/content/Audio Trimmed.mp3'


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
num_speakers = 2

language = 'English'

model_size = 'large'

model_name = model_size
if language == 'English' and model_size != 'large':
  model_name += '.en'


In [4]:
!pip install -q git+https://github.com/openai/whisper.git > /dev/null
!pip install -q git+https://github.com/pyannote/pyannote-audio > /dev/null

import whisper
import datetime

import subprocess

import torch


In [5]:
import pyannote.audio
from pyannote.audio.pipelines.speaker_verification import PretrainedSpeakerEmbedding

# Try to use the GPU, if available and driver is correctly installed. Otherwise, default to CPU.
# try:
#   embedding_model = PretrainedSpeakerEmbedding(
#       "speechbrain/spkrec-ecapa-voxceleb",
#       device=torch.device("cuda"))
# except RuntimeError:
#   print("No GPU found or driver issue. Defaulting to CPU.")
embedding_model = PretrainedSpeakerEmbedding(
    "speechbrain/spkrec-ecapa-voxceleb",
    device=torch.device("cpu"))


from pyannote.audio import Audio
from pyannote.core import Segment

import wave
import contextlib

from sklearn.cluster import AgglomerativeClustering
import numpy as np

In [6]:
if file_path[-3:] != 'wav':
  subprocess.call(['ffmpeg', '-i', file_path, 'audio.wav', '-y'])
  file_path = 'audio.wav'

In [7]:
model = whisper.load_model(model_size)

In [8]:
result = model.transcribe(file_path)
segments = result["segments"]

In [9]:
with contextlib.closing(wave.open(file_path,'r')) as f:
  frames = f.getnframes()
  rate = f.getframerate()
  duration = frames / float(rate)

In [10]:
audio = Audio()

def segment_embedding(segment):
  start = segment["start"]
  # Whisper overshoots the end timestamp in the last segment
  end = min(duration, segment["end"])
  clip = Segment(start, end)
  waveform, sample_rate = audio.crop(file_path, clip)
  return embedding_model(waveform[None])

In [11]:
import numpy as np
import torch

audio = Audio()

def segment_embedding(segment):
    start = segment["start"]
    # Whisper overshoots the end timestamp in the last segment
    end = min(duration, segment["end"])
    clip = Segment(start, end)
    waveform, sample_rate = audio.crop(file_path, clip)

    # Ensure waveform is mono
    if waveform.ndim > 1 and waveform.shape[0] > 1:
        # Convert to mono by averaging channels using PyTorch
        waveform = torch.mean(waveform, dim=0, keepdim=True)

    return embedding_model(waveform[None])

embeddings = np.zeros(shape=(len(segments), 192))
for i, segment in enumerate(segments):
    embeddings[i] = segment_embedding(segment)

embeddings = np.nan_to_num(embeddings)


In [12]:

from sklearn.cluster import AgglomerativeClustering
import datetime

# Assuming embeddings and segments are already defined
clustering = AgglomerativeClustering(n_clusters=num_speakers).fit(embeddings)
labels = clustering.labels_

# Map cluster labels to speaker names
speaker_names = {0: 'Curtis', 1: 'Amy'}
for i in range(len(segments)):
    segments[i]["speaker"] = speaker_names.get(labels[i], 'Unknown')

def time(secs):
    return datetime.timedelta(seconds=round(secs))

with open("transcript.txt", "w") as f:
    for i, segment in enumerate(segments):
        if i == 0 or segments[i - 1]["speaker"] != segment["speaker"]:
            f.write("\n" + segment["speaker"] + ' ' + str(time(segment["start"])) + '\n')
        f.write(segment["text"][1:] + ' ')


In [13]:
# Load the contents of the .txt file
with open('transcript.txt', 'r', encoding='utf-8') as file:
    file_content = file.read()

print(file_content)


Curtis 0:00:00
Hi everybody, it's Curtis and I'm here with Amy from Real English Conversations and we're going to talk about driving in Canada and North America. And it all starts with getting your learner's license, right Amy? 
Amy 0:00:17
Yeah, definitely. In order to be on the road behind the wheel of a vehicle, you've got to have a license. And the first thing that you do is you actually, at least in Canada, we do things in a very specific way. We have something called a graduated licensing program. I'm sure that the United States does it differently. But you pick up this book and it has all the rules of the road. 
Curtis 0:00:45
Yeah, and you have to study it because you have to write a written test or something like that. 
Amy 0:00:52
Yeah, it's been a long time, right Curtis? 
Curtis 0:00:54
It's been a very long time for me, yeah. 
Amy 0:01:00
So, yeah, you have to go in. There's a test that makes sure that obviously you've studied the book, you know about the rules of making 

# **Speech Summarization**




In [14]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [15]:
!pip install transformers

from transformers import pipeline

# Initialize the summarizer pipeline
summarizer = pipeline("summarization")

# Load the contents of the .txt file
with open('transcript.txt', 'r') as file:
    file_content = file.read()

# Remove occurrences of "SPEAKER 1", "SPEAKER 2", "SPEAKER 3"
# content_cleaned = file_content.replace('SPEAKER 1', '').replace('SPEAKER 2', '').replace('SPEAKER 3', '')

# Generate the summary
summary = summarizer(file_content, max_length=150, min_length=40, do_sample=False)

# Save the summary to a new file
with open('summary.txt', 'w', encoding='utf-8') as summary_file:
    summary_file.write(summary[0]['summary_text'])

# Print the summary
print("\nSummary of the Conversation:\n", summary[0]['summary_text'])


No model was supplied, defaulted to sshleifer/distilbart-cnn-12-6 and revision a4f8f3e (https://huggingface.co/sshleifer/distilbart-cnn-12-6).
Using a pipeline without specifying a model name and revision in production is not recommended.
Hardware accelerator e.g. GPU is available in the environment, but no `device` argument is passed to the `Pipeline` object. Model will be on CPU.



Summary of the Conversation:
  In Canada, at least in Canada, we do things in a very specific way. We have something called a graduated licensing program. I'm sure that the United States does it differently. But you pick up this book and it has all the rules of the road. You have to study it because you have to write a written test .


# **Speech Translation to Arabic**


In [16]:
pip install googletrans==4.0.0-rc1

## **Summary Translation**

In [17]:
from googletrans import Translator

# Initialize the Translator
translator = Translator()

# Load the contents of the .txt file
with open('summary.txt', 'r', encoding='utf-8') as file:
    file_content = file.read()

# Translate the content to Arabic
translated = translator.translate(file_content, dest='ar')

# Save the Translated Transcripts to a new file
with open('summary_arabic.txt', 'w', encoding='utf-8') as output_file2:
    output_file2.write(translated.text)  # Use translated.text to get the string

# Print the original and translated content (optional)
print("Original Summary:\n", file_content)
print("\nTranslated Summary:\n", translated.text)


Original Summary:
  In Canada, at least in Canada, we do things in a very specific way. We have something called a graduated licensing program. I'm sure that the United States does it differently. But you pick up this book and it has all the rules of the road. You have to study it because you have to write a written test .

Translated Summary:
 في كندا ، على الأقل في كندا ، نفعل الأشياء بطريقة محددة للغاية.لدينا شيء يسمى برنامج الترخيص المتدرج.أنا متأكد من أن الولايات المتحدة تفعل ذلك بشكل مختلف.لكنك تلتقط هذا الكتاب ولديه جميع قواعد الطريق.عليك أن تدرسها لأنه يجب عليك كتابة اختبار كتابي.


## **Transcript Translation**

In [18]:
from googletrans import Translator

# Initialize the Translator
translator = Translator()

# Load the contents of the .txt file
with open('transcript.txt', 'r', encoding='utf-8') as file:
    file_content = file.read()

# Translate the content to Arabic
translated = translator.translate(file_content, dest='ar')

# Save the Translated Transcripts to a new file
with open('transcript_arabic.txt', 'w', encoding='utf-8') as output_file2:
    output_file2.write(translated.text)  # Use translated.text to get the string

# Print the original and translated content (optional)
print("\nTranslated Trnascripts:\n", translated.text)



Translated Trnascripts:
 كورتيس 0:00:00
مرحباً بالجميع ، إنه كورتيس وأنا هنا مع إيمي من محادثات اللغة الإنجليزية الحقيقية وسنتحدث عن القيادة في كندا وأمريكا الشمالية.ويبدأ كل شيء بالحصول على رخصة المتعلم ، أليس كذلك؟
ايمي 0:00:17
نعم بالتأكيد.لكي تكون على الطريق خلف عجلة السيارة ، يجب أن يكون لديك ترخيص.وأول ما تفعله هو أنك في الواقع ، على الأقل في كندا ، نفعل الأشياء بطريقة محددة للغاية.لدينا شيء يسمى برنامج الترخيص المتدرج.أنا متأكد من أن الولايات المتحدة تفعل ذلك بشكل مختلف.لكنك تلتقط هذا الكتاب ولديه جميع قواعد الطريق.
كورتيس 0:00:45
نعم ، وعليك أن تدرسه لأنك يجب أن تكتب اختبارًا مكتوبًا أو شيء من هذا القبيل.
ايمي 0:00:52
نعم ، لقد مر وقت طويل ، كورتيس صحيح؟
كورتيس 0:00:54
لقد مر وقت طويل جدًا بالنسبة لي ، نعم.
ايمي 0:01:00
لذا ، نعم ، عليك أن تدخل. هناك اختبار يتأكد من الواضح أنك درست الكتاب ، أنت تعرف عن قواعد جعل منعطفًا يمينيًا ، على سبيل المثال ، عندما تحتاج إلى العائد ، عندما تحتاجللتوقف ، كل هذا النوع من الأشياء.
كورتيس 0:01:19
موازنة موازنة؟
ايمي 0:01:20
نعم ، ثم إذا اجتاز